# Optimization for medical image segmentation with 2D U-Net on Intel(R) Xeon CPUs

### 1. Brain MRI scan
Magnetic resonance imaging (MRI) of the brain is a safe and painless test that uses a magnetic field and radio waves to produce detailed images of the brain and the brain stem. An MRI differs from a CAT scan (also called a CT scan or a computed axial tomography scan) because it does not use radiation.

An MRI scanner consists of a large doughnut-shaped magnet that often has a tunnel in the center. Patients are placed on a table that slides into the tunnel. Some centers have open MRI machines that have larger openings and are helpful for patients with claustrophobia. MRI machines are located in hospitals and radiology centers.

During the exam, radio waves manipulate the magnetic position of the atoms of the body, which are picked up by a powerful antenna and sent to a computer. The computer performs millions of calculations, resulting in clear, cross-sectional black and white images of the body. These images can be converted into three-dimensional (3-D) pictures of the scanned area. This helps pinpoint problems in the brain and the brain stem when the scan focuses on those areas.

**Reference:** https://kidshealth.org/en/parents/mri-brain.html

<table><tr><td><img src='https://github.com/IntelAI/unet/raw/master/3D/images/BRATS_152_img3D.gif'></td><td><img src='https://github.com/IntelAI/unet/blob/master/3D/images/BRATS_195_img.gif?raw=true'></td></tr></table>

**Reference:** https://github.com/IntelAI/unet

### 2. U-Net for brain segmentation

U-Net implementation in TensorFlow for FLAIR abnormality segmentation in brain MRI based on a deep learning segmentation algorithm used in [Association of genomic subtypes of lower-grade gliomas with shape features automatically extracted by a deep learning algorithm](https://doi.org/10.1016/j.compbiomed.2019.05.002).

```latex
@article{buda2019association,
  title={Association of genomic subtypes of lower-grade gliomas with shape features automatically extracted by a deep learning algorithm},
  author={Buda, Mateusz and Saha, Ashirbani and Mazurowski, Maciej A},
  journal={Computers in Biology and Medicine},
  volume={109},
  year={2019},
  publisher={Elsevier},
  doi={10.1016/j.compbiomed.2019.05.002}
}
```

Topology structured as the following:
<img src='https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/unet.png'>

**Reference:** https://github.com/mateuszbuda/brain-segmentation-pytorch

### 3. Dataset
We use [brain tumor segmentation (BraTS) subset](https://drive.google.com/file/d/1A2IU8Sgea1h3fYLpYtFb2v7NYdMjvEhU/view?usp=sharing) of the [Medical Segmentation Decathlon](http://medicaldecathlon.com/) dataset. The dataset has the [Creative Commons Attribution-ShareAlike 4.0 International license](https://creativecommons.org/licenses/by-sa/4.0/).

Please follow instructions [here](https://github.com/IntelAI/unet/blob/master/2D/00_Prepare-Data.ipynb) to prepare the dataset.

### 4. Let's do coding!

#### 4.1 Import required packages

In [43]:
%matplotlib inline
import os
import numpy as np
import tensorflow as tf
import keras as K
import h5py
import time
import matplotlib.pyplot as plt

from data import load_data
from model import unet

import sys; sys.argv=['']; del sys
from argparser import args

#### 4.2 Check TensorFlow version, and do sanity check

In [4]:
print ("We are using Tensorflow version", tf.__version__,\
       "with Intel(R) oneDNN", "enabled" if tf.pywrap_tensorflow.IsMklEnabled() else "disabled",)

We are using Tensorflow version 1.15.2 with Intel(R) oneDNN enabled


#### 4.3 Define the DICE coefficient and loss function
The Sørensen–Dice coefficient is a statistic used for comparing the similarity of two samples. Given two sets, X and Y, it is defined as

\begin{equation}
Dice = \frac{2|X\cap Y|}{|X|+|Y|}
\end{equation}

In [ ]:
def calc_dice(target, prediction, smooth=0.01):
    """
    Sorensen Dice coefficient
    """
    prediction = np.round(prediction)

    numerator = 2.0 * np.sum(target * prediction) + smooth
    denominator = np.sum(target) + np.sum(prediction) + smooth
    coef = numerator / denominator

    return coef

def calc_soft_dice(target, prediction, smooth=0.01):
    """
    Sorensen (Soft) Dice coefficient - Don't round preictions
    """
    numerator = 2.0 * np.sum(target * prediction) + smooth
    denominator = np.sum(target) + np.sum(prediction) + smooth
    coef = numerator / denominator

    return coef

#### 4.5 Load images

In [5]:
data_path = os.path.join("../../data/decathlon/144x144/")
data_filename = "Task01_BrainTumour.h5"
hdf5_filename = os.path.join(data_path, data_filename)
imgs_train, msks_train, imgs_validation, msks_validation, imgs_testing, msks_testing = load_data(hdf5_filename)
imgs_warmup=imgs_testing[:500]
imgs_infere=imgs_testing[500:2500]
print("Number of imgs_warmup: {}".format(imgs_warmup.shape[0]))
print("Number of imgs_infere: {}".format(imgs_infere.shape[0]))

Training image dimensions:   (58464, 144, 144, 4)
Training mask dimensions:    (58464, 144, 144, 1)
Validation image dimensions: (4608, 144, 144, 4)
Validation mask dimensions:  (4608, 144, 144, 1)
Testing image dimensions: (6624, 144, 144, 4)
Testing mask dimensions:  (6624, 144, 144, 1)
Number of imgs_warmup: 500
Number of imgs_infere: 2000


#### 4.5 Load model

In [6]:
unet_model = unet()
model = unet_model.load_model(os.path.join("./output/unet_model_for_decathlon.hdf5"))

Data format = channels_last
Instructions for updating:
If using Keras pass *_constraint arguments to layers.





#### 4.4 Define function to inference on input images and plot results out

In [ ]:
def plot_results(model, imgs_validation, msks_validation, img_no):
    img = imgs_validation[[img_no], ]
    msk = msks_validation[[img_no], ]
    
    pred_mask = model.predict(img, verbose=1, steps=None)

    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img[0, :, :, 0], cmap="bone", origin="lower")
    plt.axis("off")
    plt.title("MRI Input", fontsize=20)
    plt.subplot(1, 3, 2)
    plt.imshow(msk[0, :, :, 0], origin="lower")
    plt.axis("off")
    plt.title("Ground truth", fontsize=20)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask[0, :, :, 0], origin="lower")
    plt.axis("off")
    plt.title("Prediction\nDice = {:.4f}".format(calc_dice(pred_mask, msk)), fontsize=20)
    plt.tight_layout()

#### 4.6 Run inference and plot

In [ ]:
df = h5py.File("../../data/decathlon/144x144/Task01_BrainTumour.h5", "r")
imgs_testing = df["imgs_validation"]
msks_testing = df["msks_validation"]
files_testing = df["testing_input_files"]
indicies_validation = [40,61,400,1100,4385]
for idx in indicies_validation:
    plot_results(model, imgs_validation, msks_validation, idx)

#### Benchmark

In [79]:
def do_benchmark(batch_size=32):
    if 'OMP_NUM_THREADS' in os.environ:
        print(os.environ['OMP_NUM_THREADS'])
    model.predict(imgs_warmup, batch_size, verbose=1, steps=None)
    t0 = time.time()
    model.predict(imgs_infere, batch_size, verbose=1, steps=None)
    t1 = time.time()
    lat=(t1-t0)/imgs_infere.shape[0]
    thr=1/lat
    print('Latency: {}ms; Throughput: {}fps'.format(lat*1000, thr*batch_size))

In [80]:
def setEnv():
    os.environ['OMP_NUM_THREADS']='24'
    os.environ["KMP_BLOCKTIME"]='0'
    os.environ["KMP_AFFINITY"]="granularity=core,compact,1,0"
    os.environ["KMP_SETTINGS"]="0"

    os.environ["INTRA_THREADS"]='24'
    os.environ["INTER_THREADS"]='1'

def unsetEnv():
    if 'OMP_NUM_THREADS' in os.environ:
        del os.environ['OMP_NUM_THREADS']
    if 'KMP_BLOCKTIME' in os.environ:
        del os.environ["KMP_BLOCKTIME"]
    if 'KMP_AFFINITY' in os.environ:
        del os.environ["KMP_AFFINITY"]
    if 'KMP_SETTINGS' in os.environ:
        del os.environ["KMP_SETTINGS"]

    if 'INTRA_THREADS' in os.environ:
        del os.environ["INTRA_THREADS"]
    if 'INTER_THREADS' in os.environ:
        del os.environ["INTER_THREADS"]

batch_size = 1
print("default configuration")
unsetEnv()
do_benchmark(batch_size)
print("customized configuration")
setEnv()
do_benchmark(batch_size)
unsetEnv()

default configuration
 807/2000 [===========>..................] - ETA: 18s

KeyboardInterrupt: 

In [73]:
def test():
    print(os.environ['TEST'])

In [74]:
def setEnv():
    os.environ['TEST']='1'
setEnv()
test()

1


Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

SPDX-License-Identifier: EPL-2.0
